In [57]:
# import
import requests
import psycopg2
import psycopg2.extras
import datetime
from datetime import datetime
from pandas import DataFrame
import json
import calendar

import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
import cufflinks as cf
tls.set_credentials_file(username='kwarodom', api_key='ljMtxORMkyx0JmiDwIyT')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
%matplotlib inline

In [58]:
data_set = pd.read_csv("Indoor_1_3_2016.csv", parse_dates = True, index_col = 0, skiprows=[0,1])
data_set.head()

,Timezone : America/New_York,Temperature,Humidity,CO2,Noise,Pressure,TemperatureF
Timestamp,,,,,,,
1434254504,6/14/15 0:01,24.5,68,229,52,1009.2,76.1
1434254807,6/14/15 0:06,24.5,68,227,51,1009.3,76.1
1434255109,6/14/15 0:11,24.5,68,228,52,1009.3,76.1
1434255412,6/14/15 0:16,24.5,68,234,53,1009.3,76.1
1434255715,6/14/15 0:21,24.5,68,228,52,1009.3,76.1


In [59]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2001 entries, 1434254504 to 1434859035
Data columns (total 7 columns):
Timezone : America/New_York    2001 non-null object
Temperature                    2001 non-null float64
Humidity                       2001 non-null int64
CO2                            2001 non-null int64
Noise                          2001 non-null int64
Pressure                       2001 non-null float64
TemperatureF                   2001 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 125.1+ KB


In [60]:
data_set.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Temperature,2001.0,23.788156,0.566370,19.60,23.5,23.70,24.10,25.90
Humidity,2001.0,68.777111,3.179512,60.00,67.0,69.00,72.00,74.00
CO2,2001.0,235.133933,35.926286,178.00,214.0,224.00,244.00,487.00
Noise,2001.0,52.843578,2.783706,50.00,52.0,52.00,53.00,80.00
Pressure,2001.0,1008.565767,2.133041,1001.30,1007.3,1008.80,1009.80,1013.30
TemperatureF,2001.0,74.818681,1.019465,67.28,74.3,74.66,75.38,78.62


In [61]:
y = data_set["Timezone : America/New_York"]
z = [x[:5] + '20' + x[5:] for x in y]
print(type(z))
data_set["Timezone : America/New_York"] = z
# x = x[:5] + '20' + x[5:]
# print(x)
# print(x)
# print(x[5:8])
# x = x[:5] + '20' + x[5:]
# print(x)
# print(x[5:8])
# x[5:8] = '20'+x[5:8]
# # print(x)
# y = pd.to_datetime('6/14/15', format='%m/%d/%Y')
# print(y)
# print(type(y)) 
data_set.head()

<class 'list'>


,Timezone : America/New_York,Temperature,Humidity,CO2,Noise,Pressure,TemperatureF
Timestamp,,,,,,,
1434254504,6/14/2015 0:01,24.5,68,229,52,1009.2,76.1
1434254807,6/14/2015 0:06,24.5,68,227,51,1009.3,76.1
1434255109,6/14/2015 0:11,24.5,68,228,52,1009.3,76.1
1434255412,6/14/2015 0:16,24.5,68,234,53,1009.3,76.1
1434255715,6/14/2015 0:21,24.5,68,228,52,1009.3,76.1


In [62]:
converted_data = pd.to_datetime(data_set["Timezone : America/New_York"], format='%m/%d/%Y %H:%M')
converted_data.head()

Timestamp
1434254504   2015-06-14 00:01:00
1434254807   2015-06-14 00:06:00
1434255109   2015-06-14 00:11:00
1434255412   2015-06-14 00:16:00
1434255715   2015-06-14 00:21:00
Name: Timezone : America/New_York, dtype: datetime64[ns]

In [63]:
data_set["Timezone : America/New_York"] = converted_data

In [64]:
import datetime
from datetime import timedelta
start_date = datetime.datetime.now().date() - timedelta(days=983)
end_date = datetime.datetime.now().date() - timedelta(days=982)
print(start_date)
print(end_date)

2015-06-15
2015-06-16


In [65]:
mask = (data_set["Timezone : America/New_York"] > start_date) & (data_set["Timezone : America/New_York"] <= end_date)
print(len(data_set.loc[mask]))
data_to_use = data_set.loc[mask]
data_to_use

286


,Timezone : America/New_York,Temperature,Humidity,CO2,Noise,Pressure,TemperatureF
Timestamp,,,,,,,
1434341028,2015-06-15 00:03:00,24.4,72,218,51,1009.9,75.92
1434341331,2015-06-15 00:08:00,24.3,72,224,52,1009.7,75.74
1434341633,2015-06-15 00:13:00,24.3,72,228,52,1009.9,75.74
1434341935,2015-06-15 00:18:00,24.3,72,219,51,1009.7,75.74
1434342238,2015-06-15 00:23:00,24.3,72,222,51,1009.6,75.74
1434342540,2015-06-15 00:29:00,24.3,72,223,51,1009.7,75.74
1434342843,2015-06-15 00:34:00,24.3,72,225,52,1009.6,75.74
1434343146,2015-06-15 00:39:00,24.3,72,219,52,1009.5,75.74
1434343448,2015-06-15 00:44:00,24.3,72,210,51,1009.6,75.74


In [66]:
data_to_use.columns

Index(['Timezone : America/New_York', 'Temperature', 'Humidity', 'CO2',
       'Noise', 'Pressure', 'TemperatureF'],
      dtype='object')

In [67]:
trace1 = go.Scatter(
    x=data_to_use["Timezone : America/New_York"],
    y=data_to_use["Temperature"],
    name='Temperature'
)

trace2 = go.Scatter(
    x=data_to_use["Timezone : America/New_York"],
    y=data_to_use["Humidity"],
    name='Humidity',
    yaxis='y2'
)

data = [trace1, trace2]
layout = go.Layout(
    xaxis=dict(
        title='time'
    ),
    yaxis=dict(
        title='Temperature'
    ),
    yaxis2=dict(
        title='Humidity',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='test/test')

In [69]:
import plotly.plotly as py
import plotly.graph_objs as go

# Create random data with numpy
import numpy as np

N = 1000
random_x = np.random.randn(N)
random_y = np.random.randn(N)

# Create a trace
trace = go.Scatter(
    x = data_to_use["Temperature"],
    y = data_to_use["Humidity"],
    mode = 'markers',
    name = 'Temperature vs Humidity'
)

trace1 = go.Scatter(
    x = [21.5,25],
    y = [98,0],
    mode = 'lines+markers',
    name = 'too-cool'
)

trace2 = go.Scatter(
    x = [26.3,32],
    y = [100,0],
    mode = 'lines+markers',
    name = 'too-hot'
)

data = [trace, trace1, trace2]

# Plot and embed in ipython notebook!
py.iplot(data, filename='basic-scatter')

# or plot with: plot_url = py.plot(data, filename='basic-line')

In [14]:
import plotly.plotly as py
import plotly.graph_objs as go

# Create random data with numpy
import numpy as np

N = 100
random_x = np.linspace(0, 1, N)
random_y0 = np.random.randn(N)+5
random_y1 = np.random.randn(N)
random_y2 = np.random.randn(N)-5

# Create traces
trace0 = go.Scatter(
    x = random_x,
    y = random_y0,
    mode = 'markers',
    name = 'markers'
)
trace1 = go.Scatter(
    x = random_x,
    y = random_y1,
    mode = 'lines+markers',
    name = 'lines+markers'
)
trace2 = go.Scatter(
    x = random_x,
    y = random_y2,
    mode = 'lines',
    name = 'lines'
)

data = [trace0, trace1, trace2]
py.iplot(data, filename='scatter-mode')